In [ ]:
! pip install xgboost
! pip install pandas
! pip install scikit-learn
! pip install pyarrow

In [1]:
################################ Libraries ################################
from Input_Variables.read_vars import xgboost_regression_model_storage_location, \
                                      linear_regression_model_storage_location, \
                                      random_forest_regression_model_storage_location, \
                                      factorization_machines_regression_model_storage, \
                                      xgboost_classification_model_storage_location, \
                                      logistic_regression_classification_model_storage_location, \
                                      random_forest_classification_model_storage_location, \
                                      multilayer_perceptron_classification_model_storage_location, \
                                      naive_bayes_classification_model_storage_location, \
                                      factorization_machine_classification_model_storage_location, \
                                      random_seed
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.scaling_pipeline import Feature_Transformations
from Model_Creation.regression_models import Create_Regression_Models
from Model_Creation.classification_models import Create_Classification_Models
import os


################################ Read In Modules ################################
reading_data=Reading_Data()
feature_transformations=Feature_Transformations()
create_regression_models=Create_Regression_Models()
create_classification_models=Create_Classification_Models()


################################ Regression, Classification, Or Both ################################
train_regression=False
train_classification=True


################################ Read In Data ################################
# Training Summary Stats Data
training_files=list(map(lambda x: os.path.join(os.path.abspath('/cephfs/summary_stats/all_train_bool_updated'),x),
                                               os.listdir('/cephfs/summary_stats/all_train_bool_updated')))
training_files=[i for i in training_files if not ('.crc' in i or 'SUCCESS' in i)]


# Cross Validation Summary Stats Data
val_files=list(map(lambda x: os.path.join(os.path.abspath('/cephfs/summary_stats/all_val_bool_updated'), x),
                                          os.listdir('/cephfs/summary_stats/all_val_bool_updated')))
val_files=[i for i in val_files if not ('.crc' in i or 'SUCCESS' in i)]


# Calling DataFrames
summary_stats_train=reading_data.read_in_all_summary_stats(file_list=training_files)
summary_stats_val=reading_data.read_in_all_summary_stats(file_list=val_files)


################################ Combine Train and Cross Validation ################################
df_train_val_combined=summary_stats_train.union(summary_stats_val)
df_train_val_combined.show(2)
print((df_train_val_combined.count(), len(df_train_val_combined.columns)))


################################ Stages: Scaling Using Custom Transformer ################################
pipeline_transformation_stages=feature_transformations.numerical_scaling(df=df_train_val_combined)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/31 18:53:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/31 18:53:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-----+-----+-----------------+----------------+-------------+-------------+------------------+------------------+------------------+------+-----+-----+--------------------+--------------------+------------------+-----------------+----------+----------+---------------+------------+------+----------+--------+------------------+-------------------------+------------+-------------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|NumId|Chunk|ShortTermVariance|LongTermVariance|VarianceRatio|SampleEntropy|PermutationEntropy|              Mean|            StdDev|Median|  Min|  Max|        AvgFirstDiff|          AvgS

In [2]:
from xgboost.spark import SparkXGBClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import FMClassifier

from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [3]:
features_col="features"
label_name="target"
prediction_column_name="prediction"
num_folds=3

In [9]:
mapping= {0: 0, 1: 1, -1: 2}
df_train_val_combined=df_train_val_combined.replace(to_replace=mapping, subset=['target'])

In [10]:
layers = [4, 2, 2]

In [11]:
mlp=FMClassifier(featuresCol=features_col, 
               labelCol=label_name,
                seed=random_seed)

In [12]:
pipeline_transformation_stages.pop()

FMClassifier_55c46d212900

In [13]:
pipeline_transformation_stages

[ColumnScaler_a7a8f908bd2e, VectorAssembler_6e973433668e]

In [14]:
pipeline_transformation_stages.append(mlp)
pipeline=Pipeline(stages=pipeline_transformation_stages)

In [15]:
model=pipeline.fit(df_train_val_combined)

23/05/31 18:54:57 WARN BlockManager: Putting block rdd_165_3 failed due to exception java.lang.IllegalArgumentException: requirement failed: Classifier was given dataset with invalid label 2.0.  Labels must be integers in range [0, 2)..
23/05/31 18:54:57 WARN BlockManager: Putting block rdd_165_0 failed due to exception java.lang.IllegalArgumentException: requirement failed: Classifier was given dataset with invalid label 2.0.  Labels must be integers in range [0, 2)..
23/05/31 18:54:57 WARN BlockManager: Block rdd_165_3 could not be removed as it was not found on disk or in memory
23/05/31 18:54:57 WARN BlockManager: Block rdd_165_0 could not be removed as it was not found on disk or in memory
23/05/31 18:54:57 ERROR Executor: Exception in task 0.0 in stage 13.0 (TID 54)
java.lang.IllegalArgumentException: requirement failed: Classifier was given dataset with invalid label 2.0.  Labels must be integers in range [0, 2).
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.m

Py4JJavaError: An error occurred while calling o353.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 54) (jupyter-cmonsivais-40ucsd-2eedu executor driver): java.lang.IllegalArgumentException: requirement failed: Classifier was given dataset with invalid label 2.0.  Labels must be integers in range [0, 2).
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.classification.Classifier.validateLabel(Classifier.scala:122)
	at org.apache.spark.ml.classification.Classifier.$anonfun$extractLabeledPoints$1(Classifier.scala:98)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:224)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:302)
	at org.apache.spark.storage.BlockManager.

In [ ]:
ml_df=df_train_val_combined.withColumn("foldCol", df_train_val_combined.NumId % num_folds)

evaluator_logloss=MulticlassClassificationEvaluator(metricName='logLoss',
                                                    labelCol=label_name,
                                                    predictionCol=prediction_column_name)
paramGrid=ParamGridBuilder().build()

In [ ]:
crossval=CrossValidator(estimator=xgb,
                        evaluator=evaluator_logloss,
                        estimatorParamMaps=paramGrid,
                        foldCol='foldCol',
                        collectSubModels=False)

In [ ]:
pipeline_transformation_stages.append(crossval)
pipeline=Pipeline(stages=pipeline_transformation_stages)

model=pipeline.fit(ml_df)

In [ ]:
        location_counter=0
        model_types=['XGBoost', 'Logistic_Regression', 'Random_Forest', 'Multilayer_Perceptron', 'Naive_Bayes', 'Factorization_Machine']
        model_mapping={'XGBoost': SparkXGBClassifier(features_col=self.features_col, 
                                                     label_col=self.label_name,
                                                     random_state=random_seed,
                                                     use_gpu=True),
                       
                       'Logistic_Regression': LogisticRegression(featuresCol=self.features_col, 
                                                                 labelCol=self.label_name,
                                                                 standardization=False),
                       
                       'Random_Forest': RandomForestClassifier(featuresCol=self.features_col, 
                                                               labelCol=self.label_name,
                                                               seed=random_seed),
                       
                       'Multilayer_Perceptron': MultilayerPerceptronClassifier(featuresCol=self.features_col, 
                                                                               labelCol=self.label_name,
                                                                               seed=random_seed),
                       
                       'Naive_Bayes': NaiveBayes(featuresCol=self.features_col, 
                                                 labelCol=self.label_name),
                       
                       'Factorization_Machine': FMClassifier(featuresCol=self.features_col, 
                                                             labelCol=self.label_name,
                                                             seed=random_seed)
                      }
        
        ml_df=ml_df.withColumn("foldCol", ml_df.NumId % num_folds)
        
        evaluator_logloss=MulticlassClassificationEvaluator(metricName='logLoss',
                                                            labelCol=self.label_name,
                                                            predictionCol=self.prediction_column_name)
        paramGrid=ParamGridBuilder().build()
        
        for model_type in model_types:
            if location_counter > 0:
                stages.pop()
                print(f'Currently on {model_type} Model')
            else:
                print(f'Currently on {model_type} Model')
            crossval=CrossValidator(estimator=model_mapping[model_type],
                                    evaluator=evaluator_logloss,
                                    estimatorParamMaps=paramGrid,
                                    foldCol='foldCol',
                                    collectSubModels=False)

            print('Cross Validation Occuring')
            stages.append(crossval)
            pipeline=Pipeline(stages=stages)

            model=pipeline.fit(ml_df)

            model.write().overwrite().save(classification_models_storage_locations[location_counter])
            print(f'Model Saved to {classification_models_storage_locations[location_counter]}')
            location_counter+=1